# Fine-tuning LLaMa2 🦙

In this demonstration, we show how to use the Predibase SDK to fine tune LLaMa2-7B. We give a detailed walkthrough of all the components required for setup and how to work in sync with the UI.

## Authentication 🔐

The first step is to sign into Predibase. 

- If you do not have a Predibase account set up yet, you may sign up for a free account [here](https://predibase.com/free-trial)
- If you already have an account, navigate to Settings -> My Profile and generate a new API token.
- Finally, plug in the generated API token in the code below to authenticate.

In [1]:
import pandas as pd
from predibase import PredibaseClient

%load_ext autoreload
%autoreload 2

pc = PredibaseClient(
    token="YOUR TOKEN HERE"
)

Connected to Predibase as User(id=3d4f91fd-3bea-48d0-9f86-1385c51b521a, 
username=connor_mccormick_predibase)

## Dataset 📄

Next we'll get the dataset we need to train the model. For this demonstration, we will be using the [Consumer Complaints dataset](https://www.kaggle.com/datasets/selener/consumer-complaint-database) from Kaggle. We have already uploaded this dataset to Predibase, so we just need to get it. If you need help connecting a dataset to Predibase, the [connections](https://docs.predibase.com/sdk-guide/connections/) and [datasets](https://docs.predibase.com/sdk-guide/datasets/) docs are a great resource!

This dataset contains real world complaints received about financial products and services. We will be fine-tuning the model to classify the issue type of written complaint.

In [2]:
consumer_complaints_dataset = pc.get_dataset(
    dataset_name="DATASET NAME IN PREDIBASE",
    connection_name="CONNECTION NAME IN PREDIBASE"
)
consumer_complaints_dataset

Dataset(id=62, name=consumer_complaints_narrative, object_name=s3://datasets.us-west-2.predibase.com/consumer_complaints_narrative_dropna.parquet, connection_id=7, author=wael, created=2022-12-21T11:29:11.576627Z, updated=2022-12-21T11:29:11.576627Z)

## Engine 🚂

At Predibase, engines are our solution to common compute and infrastructure pain points that everyone runs into while training models. These are problems like:
- Encountering Out of Memory errors due to insufficient compute
- Challenges distributing a model training job over multiple compute resources
- Losing progress when transient issues interrupt the training process

Predibase training engines mitigate these issues by:
- Analyzing the training job details to assign the right amount of compute
- Logic to distribute the training job over the assigned compute resources
- Retry logic when things go wrong

With this in mind, we will select the engine we want to use for training.

In [3]:
train_engine = pc.get_engine("train_engine")
train_engine

Engine(id=50, name=train_engine, status=suspended, type=kubernetes, service_type=batch, template_id=8, environment_id=1, auto_suspend_seconds=0, auto_resume=True)

## Model Training 🏁

Now we can kick off our fine-tuning job! 

All we need to do is specify the LLM that we will be fine tuning and create a template telling the LLM the task to complete. Then we call `llm.finetune()` and we're on our way.


You can follow the link in the output to track the model's progress in the UI!

In [4]:
llm = pc.LLM("hf://meta-llama/Llama-2-7b-hf")

In [5]:
consumer_complaints_finetuning_template = """
    Given the following real world consumer complaint received about a financial product or service, classify the issue type: 
    Consumer Complaint: {Consumer complaint narrative} 
    Product: {Product} 
    Sub-Product: {Sub-product} 
    Issue Type:
"""

In [9]:
fine_tuning_job = llm.finetune(
    template=consumer_complaints_finetuning_template,
    target="Issue",
    dataset=consumer_complaints_dataset,
    engine=train_engine,
    repo="Fine-tuning Consumer Complaints"
)

Model repository [Connor] Fine-tuning Consumer Complaints already exists and new models will 
be added to it.

Check Status of Model Training Here: ]8;id=861710;https://staging.predibase.com/models/version/49471\https://staging.predibase.com/models/version/49471]8;;\

## Advanced Configuration 📝

Like everything in Predibase, you have a tremendous amount of control if you want to configure it (otherwise, we do it for you). Below is a yaml config for a fine-tuning job with some of the cool features that can improve the training process!

```
model_type: llm
base_model: meta-llama/Llama-2-7b-hf

quantization:
  bits: 4

adapter:
  type: lora

prompt:
  template: >-
    Given the following real world consumer complaint received about a financial
    product or service, classify the issue type: 
    Consumer Complaint: {Consumer complaint narrative} 
    Product: {Product} 
    Sub-Product: {Sub-product} 
    Issue Type:

input_features:
  - name: Consumer complaint narrative
    type: text

output_features:
  - name: Issue
    type: text

trainer:
  type: finetune
  learning_rate: 0.0001
  batch_size: 1
  gradient_accumulation_steps: 16
  epochs: 3
  learning_rate_scheduler:
    decay: cosine
    warmup_fraction: 0.01

preprocessing:
  sample_ratio: 0.1

```
A few of the key parameters set are outlined below:
- `model_type`: this indicates that we want to train an LLM model instead of an ECD or GBM model.
- `base_model`: this is the open source model that we will be fine-tuning
- `quantization`: this parameter specifies the level of quantization to use during the fine-tuning process. Different levels of quantization tell Predibase to use more memory efficient calculations method which allows us to fine-tune on cheaper more widely available hardware. The tradeoff is time as lower levels of quantization tend to take longer to complete the fine-tuning process.
- `adapter.type`: this indicates the method of parameter efficient fine tuning (PEFT) we want to use.
- `prompt.template`: here we are providing a prompt template that extracts the specfied values from the dataset for every row and constructs a prompt that is fed to the model. By being very explicit here, we can help the model achieve better performance.
- `preprocessing.sample_ratio`: this parameter tells Predibase to only use a subset of the dataset to train. This dataset has ~1 million rows, but we only need about 1000 to fine-tune.

For more configuration details, check out the [Ludwig LLM Docs](https://ludwig.ai/0.8/configuration/large_language_model/)!

In [4]:
consumer_complaints_config = {
    'model_type': 'llm',
    'base_model': 'meta-llama/Llama-2-7b-hf',
    'quantization': {
        'bits': 4
    },
    'adapter': {
        'type': 'lora'
    },
    'prompt': {
        'template': """
            Given the following real world consumer complaint received about a financial product or service, classify the issue type:
            Consumer Complaint: {Consumer complaint narrative}
            Product: {Product}
            Sub-Product: {Sub-product}
            Issue Type:
            """
    },
    'input_features': [
        {'name': 'Consumer complaint narrative', 'type': 'text'}
    ],
    'output_features': [
        {'name': 'Issue', 'type': 'text'}
    ],
    'trainer': {
        'type': 'finetune',
        'learning_rate': 0.0001,
        'batch_size': 1,
        'gradient_accumulation_steps': 16,
        'epochs': 3,
        'learning_rate_scheduler': {
            'decay': 'cosine', 
            'warmup_fraction': 0.01
        }
    },
    'preprocessing': {
        'sample_ratio': 0.1
    }
}

In [5]:
consumer_complaints_model = pc.create_model(
    repository_name="Fine-tuning Consumer Complaints", 
    dataset=consumer_complaints_dataset, 
    config=consumer_complaints_config, 
    engine=train_engine,
    model_description="Basic Fine-tuning LLaMa2-7B on Consumer Complaints"
)

Created model repository: <[Connor] Fine-tuning Consumer Complaints>

Training model version 1403 for model repository <[Connor] Fine-tuning Consumer 
Complaints>...

Check Status of Model Training Here: ]8;id=273474;https://app.predibase.com/models/version/3527\https://app.predibase.com/models/version/3527]8;;\